> Neural Network Weight Quantization Project for Harvard CS 2420: Computing at Scale (Fall 2024)


---

### **Setup**

---

In [ ]:
!nvidia-smi

Thu Oct 10 01:13:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              46W / 400W |   1079MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import sys
import time
import os
import math

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from copy import deepcopy

from torch.autograd import Variable

In [ ]:
# tracks the highest accuracy observed so far
best_acc = 0

def moving_average(a, n=100):
    '''Helper function used for visualization'''
    ret = torch.cumsum(torch.Tensor(a), 0)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def train(net, epoch, loader, criterion, optimizer, loss_tracker = [], acc_tracker = []):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        # update optimizer state
        optimizer.step()
        # compute average loss
        train_loss += loss.item()
        loss_tracker.append(loss.item())
        loss = train_loss / (batch_idx + 1)
        # compute accuracy
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100. * correct / total
        # Print status
        sys.stdout.write(f'\rEpoch {epoch}: Train Loss: {loss:.3f}' +
                         f'| Train Acc: {acc:.3f}')
        sys.stdout.flush()
    acc_tracker.append(acc)
    sys.stdout.flush()

def test(net, epoch, loader, criterion, loss_tracker = [], acc_tracker = []):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            loss_tracker.append(loss.item())
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            loss = test_loss / (batch_idx + 1)
            acc = 100.* correct / total
    sys.stdout.write(f' | Test Loss: {loss:.3f} | Test Acc: {acc:.3f}\n')
    sys.stdout.flush()

    # Save checkpoint
    acc = 100.*correct/total
    acc_tracker.append(acc)
    if acc > best_acc:
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [ ]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1,
               padding=1):
    '''
    A nn.Sequential layer executes its arguments in sequential order. In
    this case, it performs Conv2d -> BatchNorm2d -> ReLU. This is a typical
    block of layers used in Convolutional Neural Networks (CNNs). The
    ConvNet implementation below stacks multiple instances of this three layer
    pattern in order to achieve over 90% classification accuracy on CIFAR-10.
    '''
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding,
                  bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
        )

class ConvNet(nn.Module):
    '''
    A 9 layer CNN using the conv_block function above. Again, we use a
    nn.Sequential layer to build the entire model. The Conv2d layers get
    progressively larger (more filters) as the model gets deeper. This
    corresponds to spatial resolution getting smaller (via the stride=2 blocks),
    going from 32x32 -> 16x16 -> 8x8. The nn.AdaptiveAvgPool2d layer at the end
    of the model reduces the spatial resolution from 8x8 to 1x1 using a simple
    average across all the pixels in each channel. This is then fed to the
    single fully connected (linear) layer called classifier, which is the output
    prediction of the model.
    '''
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            conv_block(3, 32),
            conv_block(32, 32),
            conv_block(32, 64, stride=2),
            conv_block(64, 64),
            conv_block(64, 64),
            conv_block(64, 128, stride=2),
            conv_block(128, 128),
            conv_block(128, 256),
            conv_block(256, 256),
            nn.AdaptiveAvgPool2d(1)
            )

        self.classifier = nn.Linear(256, 10)

    def forward(self, x):
        '''
        The forward function is called automatically by the model when it is
        given an input image. It first applies the 8 convolution layers, then
        finally the single classifier layer.
        '''
        h = self.model(x)
        B, C, _, _ = h.shape
        h = h.view(B, C)
        return self.classifier(h)

---

### **Post-training Quantization**

---

In this section, I performed post-training quantization of weights on a multi-layer perceptron with fully connected layers to reduce its memory requirements.
I started by training a small neural network on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset. Then, I quantized its weights and verified that performance does not degrade significantly.

In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

class MNISTNet(nn.Module):
    def __init__(self, hidden=128):
      super(MNISTNet, self).__init__()

      # First 2D convolutional layer, taking in 1 input channel (image),
      # outputting 32 convolutional features, with a square kernel size of 3
      self.hidden = hidden
      self.fc1 = nn.Linear(28*28*1, self.hidden)
      self.fc2 = nn.Linear(self.hidden, 10)

    def forward(self, x):
      x = x.view(-1, 28*28)
      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc2(x)
      return F.log_softmax(x, dim=1)

def train_mnist(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test_mnist(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

model = MNISTNet()
device = torch.device("cpu")

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1_mnist = datasets.MNIST('./data_mnist', train=True, download=True,
                    transform=transform)
dataset2_mnist = datasets.MNIST('./data_mnist', train=False,
                    transform=transform)
train_loader_mnist = torch.utils.data.DataLoader(dataset1_mnist)
test_loader_mnist = torch.utils.data.DataLoader(dataset2_mnist)

optimizer = optim.Adam(model.parameters())

scheduler = StepLR(optimizer, step_size=1, gamma=.1)
for epoch in range(1, 5):
    train_mnist(model, device, train_loader_mnist, optimizer, epoch)
    test_mnist(model, device, test_loader_mnist)
    scheduler.step()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:05<00:00, 1894068.43it/s]


Extracting ./data_mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data_mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 345592.48it/s]


Extracting ./data_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data_mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3195576.12it/s]


Extracting ./data_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data_mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 10811877.85it/s]


Extracting ./data_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data_mnist/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.133334
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.000006
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.000000
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.000001
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.000435
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.000013

Test set: Average loss: 0.3227, Accuracy: 9252/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.000345
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.000000
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.000000
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.000008
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.000000
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.000000

Test set: Average loss: 0.1307, Accuracy: 9702/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.001012
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.000000
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.000000
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.000000
Train 

---
We will perform post-training quantization on the weights of a neural network. Quantization maps high-precision values to low-precision values to reduce the number of bits used to store each value. In this section we will implement two functions:
* **quantize**: compresses a vector/matrix from 32-bit float to 8-bit integer
* **dequantize**: takes the output of **quantize** and recovers the original values (possibly with some error).

**Quantization Example**

Given a vector of 32-bit floats containing the values `[400, 500, 600]`, our goal is to quantize these values so that they are representable in 8 bits. With 8 bits, we can only represent the values 0 through 255 -- but we have a problem: 400, 500 and 600 are outside of this range! To solve this, we want to map the values of the vector such that each value is *approximately* representable by values between 0 and 255. To do this we:
1. Shift the vector so that the minimum value is 0
2. Scale the vector so that every value is between 0 and 255
3. Cast each value to an (8-bit) integer

For example, suppose we are quantizing `v=[400,500,600]`.
* After shifting would have `shift=400`, `v_shifted=[0,100,200]`.
* After scaling, we would have `scale=255/200`, `shift=400`, `v_shifted_scaled=[0*255/200, 100 *255/200, 200 *255/200] = [0, 127.5, 255]`, giving us `v_q=[0, 128, 255]` after rounding.

With this process we have successfully quantized the elements of `v` to be representable by 8 bits (with additional `scale` and shift `parameters`). This yields ~4x reduction in size for large vectors!

**Dequantization Example**

To dequantize, we do the opposite:
* Given: `v_q=[0,128,255]`, `shift=400`, `scale=255/200`
* Reverse scale: `v_unscaled = v_q / scale = [0, 100.392, 200]`
* Reverse shift: `v_unscaled + 400 = [400, 500.392, 600]`

As you can see, our dequantized values are very close to our original values (`[400,500,600]`).

In [ ]:
import numpy as np
import sys

def quantize(W):
    shift = np.min(W)
    W_shift = W - shift
    scale = np.float32(255 / np.max(W_shift))
    W_quantized = np.round(W_shift * scale)
    return np.array(W_quantized, dtype=np.uint8), (shift, scale)

def dequantize(W, extra_args):
    shift, scale = extra_args
    W_unscaled = np.array(W, dtype=np.float32) / scale
    return W_unscaled + shift

In [ ]:
print("Baseline Score")
test_mnist(model, device, test_loader_mnist)

state_dict = model.state_dict()
state_dict_q = {}
for key, value in state_dict.items():
  quantized, extra_args = quantize(value.numpy())
  dequantized = dequantize(quantized, extra_args)
  state_dict_q[key] = torch.from_numpy(dequantized)

model.load_state_dict(state_dict_q)

print("Quantized performance")
test_mnist(model, device, test_loader_mnist)

Baseline Score

Test set: Average loss: 0.1217, Accuracy: 9714/10000 (97%)

Quantized performance

Test set: Average loss: 0.1217, Accuracy: 9714/10000 (97%)

